<img src="figs/eh_logo.png" style="width: 200px;">

# EmptyHeaded + Aggregations

We add basic aggregations to EmptyHeaded following the work of [Puttagunta et al](http://arxiv.org/abs/1508.07532). The key optimization that we use in EmptyHeaded which other worst-case optimal join processing engines do not use is the ability perform aggregations. We use *generalized hypertree decompositions* (GHDs) as our query plans in EmptyHeaded (thus our our equivalent of relational algebra). This allows us to treat aggregations as tropical semirings that support standard aggregation operations like $\sum,\times,\max,\min$.  

## Annotations

The sets of values in the trie can optionally be associated with data values (1-1 mapping) which are used in aggregations. We call these associated values *annotations* following the work of [Green et al.](http://dl.acm.org/citation.cfm?id=1265535). For example, a two level trie annotated with a float value represents a sparse matrix or graph with edge properties. This simple collection of operations over the trie data structure is sufficient to express a wide variety of workloads. 

## Annotations

These annotations support aggregations from
any semiring (a generalization of natural numbers equipped with a
notion of addition and multiplication). This enables EmptyHeaded to support
classic aggregations such as $SUM$, $MIN$, or $COUNT$, but also
enables \EH to support more complicated computations such as gradient
descent. To specify the annotation, one uses a semicolon in the head
of the rule, e.g. $q(x, y; z:int)$ specifies that each $x,y$ pair will
be associated with an integer value with alias $z$ similar to a GROUP
BY in SQL. In addition, the user expresses the aggregation operation
in the body of the rule. The user can specify an initialization value
as any expression over the tuples' values and constants, while common
aggregates have default values. A typical
count query is shown next.

## Loading the database

First lets load a database to run the aggregation query over.

In [1]:
import emptyheaded
emptyheaded.createDB("$EMPTYHEADED_HOME/examples/graph/data/facebook/config.json")
emptyheaded.loadDB("$EMPTYHEADED_HOME/examples/graph/data/facebook/db")

Created database with the following relations: 
	Edge(node:long,node:long)
	InvDegree(node:long)


## Composing the query

Now that the database is loaded lets write a simple aggregation query. We call this query the Barbell query or $B_{3,1}$ as it finds all 3-cliques connected by a path of length one. The syntax below shows that the annotation is first specified in the head (e.g. `m:long`) with the aggregation appearing in the body of the rule (e.g. `COUNT(*)`). Lets find out how many barbells are in our facebook dataset.

In [2]:
emptyheaded.query("Barbell(;m:long) :- Edge(a,b),Edge(b,c),Edge(a,c),Edge(a,x),Edge(x,y),Edge(y,z),Edge(x,z);m=[<<COUNT(*)>>].")

## Inspecting the result

As before the data comes back in the form of a Pandas dataframe. The caveat with annotations is that they are always the last column of the dataframe and labelled as an `annotation`.

In [5]:
emptyheaded.fetchData("Barbell")

,annotation
0,20371831447136
